In [1]:
import pandas as pd
import numpy as np
from itertools import product
from statsmodels.tsa.arima.model import ARIMA

In [2]:
sales_matrix = np.load('/Users/harshitgupta/Desktop/vs /VS-Data-Den/data_cleaning/sales_matrix.npy')

In [3]:
def calculate_mape(true_values, forecast_values):
    return np.mean(np.abs((true_values - forecast_values) / true_values)) * 100

In [4]:
def find_best_arima_order(regional_master_sales):
    best_mape = float('inf')
    best_order = None
    
    p_values = range(3)  
    d_values = range(3) 
    q_values = range(3) 

    order_combinations = product(p_values, d_values, q_values)

    for order in order_combinations:
        try:
            train_size = int(len(regional_master_sales) * 0.8)
            train_data, test_data = regional_master_sales[:train_size], regional_master_sales[train_size:]

            model = ARIMA(train_data, order=order)
            model_fit = model.fit()

            forecast = model_fit.forecast(steps=len(test_data))
            mape = calculate_mape(test_data, forecast)

            if mape < best_mape:
                best_mape = mape
                best_order = order
        except:
            continue
    
    return best_order, best_mape

In [5]:
best_overall_mape = float('inf')
best_overall_order = None

In [6]:
for regional_master_sales in sales_matrix:
    best_order, best_mape = find_best_arima_order(regional_master_sales)
    if best_mape < best_overall_mape:
        best_overall_mape = best_mape
        best_overall_order = best_order

/Users/harshitgupta/Desktop/vs /VS-Data-Den/.venv/lib/python3.9/site-packages/statsmodels/tsa/statespace/sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
/Users/harshitgupta/Desktop/vs /VS-Data-Den/.venv/lib/python3.9/site-packages/statsmodels/tsa/statespace/sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
/Users/harshitgupta/Desktop/vs /VS-Data-Den/.venv/lib/python3.9/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/var/folders/_p/ljb0l43112vc95swz8nhgv340000gn/T/ipykernel_90360/2922862027.py:2: RuntimeWarning: divide by zero encountered in divide
  return np.mean(np.abs((true_values - forecast_va

In [7]:
print(f"Best overall ARIMA order: {best_overall_order}, Overall MAPE: {best_overall_mape}%")

Best overall ARIMA order: (1, 1, 2), Overall MAPE: 8.075581089096712%
